In [10]:
import einops
from torchvision import transforms

In [2]:
from dashboard import Board

In [3]:
from transunet.model import MyTransUNet, build_the_model
import transunet.dataloader as dl

Try to load data from f/home/student/Documents/dashboard/company_target_images/
Number of images: 57
Number of masks: 57

Volume of test dataset: 57



In [4]:
out_path = "data--model=TransUNet,img_size=250px,vit_blocks=8,loss_fn=BCE"

In [5]:
import torch
torch.rand(3, 1).to("cuda")

tensor([[0.8823],
        [0.9150],
        [0.3829]], device='cuda:0')

In [6]:
model = build_the_model(device="cuda", model_path="/home/student/Documents/xrays/transunet_70epo_bce_vit8.meow")

Trainable parameters amount: 67076987


In [7]:
dl.test_dataloader

In [11]:
def analyze(source):
    if type(source) == str:
        me_arr = Image.open(source).convert('L')
        me_arr = transform(me_arr)
        me_arr = me_arr.type(torch.float32)

        me_arr = einops.rearrange(me_arr, "(b c) h w -> b c h w", b=1)
    else:
        me_arr, arr, mask = source
        
        # print(arr.shape)

        # t_output = model(arr)
        # t_output = torch.sigmoid(t_output)
        # t_output = show(t_output)
        
        
    
    me_output = model.predict(me_arr)
    
    
    if type(source) == str:
        me_output_mask = transforms.Resize(Image.open(source).size[::-1])(Image.fromarray(me_output))
        
        return Image.fromarray(
            intensity(
                (MinMaxNormalize()(np.array(Image.open(source).convert("L"))) * 255).astype(np.uint8),
                np.array(me_output_mask), (20, 20, 0)
            )
        )
    else:
        me_output_mask = me_output #transforms.Resize(source.shape)(Image.fromarray(me_output))
        me_output_mask = transforms.Resize(arr.shape[-2:])(Image.fromarray(me_output_mask))
        mask = transforms.Resize(arr.shape[-2:])(Image.fromarray(mask.numpy()[0, 0, :]))
        
        # return 
        
#         tmp = (MinMaxNormalize()(arr) * 255).numpy().astype(np.uint8)[0, ]
#         tmp = g_to_rgb(tmp)[0, :]
#         tmp = einops.rearrange(tmp, "c h w -> h w c")
            
        # tmp = intensity(
        #         (MinMaxNormalize()(arr) * 255).numpy().astype(np.uint8),
        #         np.array(me_output_mask), (120, 120, 0)
        #     )[0, 0, :, :, :] #.astype(np.uint8)
            
        # tmp = Image.fromarray(einops.rearrange(tmp, "c h w -> h w c"))
        
        # img = g_to_rgb(torch.from_numpy(show(t_images)))
        # msk = me_output_mask == 255
        
        msk = transforms.ToTensor()(me_output_mask) != 0.
        msk_original = transforms.ToTensor()(mask) != 0.

        r = dsm(image=(transforms.ToTensor()(tmp)*255).type(torch.uint8), masks=msk, alpha=0.5, colors="#20a7c8").numpy()
        r = einops.rearrange(r, "c h w -> h w c")
        r = Image.fromarray(r).convert('RGB')
                
        return r, me_output_mask, arr, msk_original[0, :], msk[0, :]

In [12]:
for i in dl.test_dataloader:
    i = [j.cuda() for j in i]
    print([j.is_cuda for j in i])
    analyze(i)
    
    break

[True, True]


NameError: name 'arr' is not defined

In [ ]:
analyze()